# Data Analysis and Pipeline Verification

- This notebook analyzes the flight price prediction dataset at different stages of the MLOps pipeline: raw, bronze, silver, and gold. The goal is to perform basic data analysis at each stage and verify if the corresponding data processing pipelines have worked as expected.
- Gets the test instances utilized for local explanation by shapley

In [3]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings("ignore")

## 1. Raw Data Analysis

We start by loading the raw dataset and performing a preliminary analysis.

In [5]:
from shared.config import core_paths, config_bronze, config_gold, config_silver

In [4]:
raw_df = pd.read_csv(core_paths.RAW_DATA_DIR / "train_validation_test" / "train.csv")
raw_df.head()

,travelCode,userCode,from,to,flightType,price,time,distance,agency,date
0,0,0,Recife (PE),Florianopolis (SC),firstClass,1434.38,1.76,676.53,FlyingDrops,2019-09-26
1,121138,1202,Florianopolis (SC),Natal (RN),firstClass,1315.27,1.84,709.37,CloudFy,2019-09-26
2,132076,1301,Florianopolis (SC),Salvador (BH),premium,1311.38,2.44,937.77,CloudFy,2019-09-26
3,28904,276,Recife (PE),Rio de Janeiro (RJ),economic,908.93,2.30,885.57,Rainbow,2019-09-26
4,88695,877,Aracaju (SE),Natal (RN),firstClass,598.61,0.46,176.33,CloudFy,2019-09-26


In [10]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 190321 entries, 0 to 190320
Data columns (total 10 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   travelCode  190321 non-null  int64  
 1   userCode    190321 non-null  int64  
 2   from        190321 non-null  object 
 3   to          190321 non-null  object 
 4   flightType  190321 non-null  object 
 5   price       190321 non-null  float64
 6   time        190321 non-null  float64
 7   distance    190321 non-null  float64
 8   agency      190321 non-null  object 
 9   date        190321 non-null  object 
dtypes: float64(3), int64(2), object(5)
memory usage: 14.5+ MB


In [11]:
raw_df.describe()

,travelCode,userCode,price,time,distance
count,190321.000000,190321.000000,190321.000000,190321.000000,190321.000000
mean,67919.412272,667.159567,955.601391,1.418343,545.875221
std,39051.352769,387.624777,362.432241,0.543563,209.246707
min,0.000000,0.000000,301.510000,0.440000,168.220000
25%,34245.000000,328.000000,672.660000,1.020000,392.760000
50%,68332.000000,662.000000,899.600000,1.460000,562.140000
75%,101679.000000,1009.000000,1222.240000,1.760000,676.530000
max,135943.000000,1339.000000,1754.170000,2.440000,937.770000


In [12]:
raw_df.describe(include="object")

,from,to,flightType,agency,date
count,190321,190321,190321,190321,190321
unique,9,9,3,3,461
top,Florianopolis (SC),Florianopolis (SC),firstClass,Rainbow,2019-09-26
freq,39868,39737,81242,81741,1335


In [13]:
raw_df.nunique()

travelCode    95430
userCode       1335
from              9
to                9
flightType        3
price           490
time             33
distance         35
agency            3
date            461
dtype: int64

In [14]:
cat_cols = list(raw_df.select_dtypes(include="object").columns)
cat_cols.remove("date")
for col in cat_cols:
    unique_cats = list(raw_df[col].unique())
    print(f"{col}:{unique_cats}")

from:['Recife (PE)', 'Florianopolis (SC)', 'Aracaju (SE)', 'Campo Grande (MS)', 'Brasilia (DF)', 'Natal (RN)', 'Sao Paulo (SP)', 'Rio de Janeiro (RJ)', 'Salvador (BH)']
to:['Florianopolis (SC)', 'Natal (RN)', 'Salvador (BH)', 'Rio de Janeiro (RJ)', 'Aracaju (SE)', 'Sao Paulo (SP)', 'Recife (PE)', 'Campo Grande (MS)', 'Brasilia (DF)']
flightType:['firstClass', 'premium', 'economic']
agency:['FlyingDrops', 'CloudFy', 'Rainbow']


### Initial Thoughts on Raw Data:
- The dataset contains a mix of numerical and categorical features.
- Column names are not standardized.


## 2. Bronze Data Analysis

In [15]:
bronze_train_df = pd.read_csv(config_bronze.BRONZE_PROCESSED_DIR / "train.csv")
bronze_train_df.head()

,travelCode,userCode,from,to,flightType,price,time,distance,agency,date
0,0,0,Recife (PE),Florianopolis (SC),firstClass,1434.38,1.76,676.53,FlyingDrops,2019-09-26
1,121138,1202,Florianopolis (SC),Natal (RN),firstClass,1315.27,1.84,709.37,CloudFy,2019-09-26
2,132076,1301,Florianopolis (SC),Salvador (BH),premium,1311.38,2.44,937.77,CloudFy,2019-09-26
3,28904,276,Recife (PE),Rio de Janeiro (RJ),economic,908.93,2.30,885.57,Rainbow,2019-09-26
4,88695,877,Aracaju (SE),Natal (RN),firstClass,598.61,0.46,176.33,CloudFy,2019-09-26


In [16]:
bronze_train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 190321 entries, 0 to 190320
Data columns (total 10 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   travelCode  190321 non-null  int64  
 1   userCode    190321 non-null  int64  
 2   from        190321 non-null  object 
 3   to          190321 non-null  object 
 4   flightType  190321 non-null  object 
 5   price       190321 non-null  float64
 6   time        190321 non-null  float64
 7   distance    190321 non-null  float64
 8   agency      190321 non-null  object 
 9   date        190321 non-null  object 
dtypes: float64(3), int64(2), object(5)
memory usage: 14.5+ MB


In [17]:
bronze_train_df.describe()

,travelCode,userCode,price,time,distance
count,190321.000000,190321.000000,190321.000000,190321.000000,190321.000000
mean,67919.412272,667.159567,955.601391,1.418343,545.875221
std,39051.352769,387.624777,362.432241,0.543563,209.246707
min,0.000000,0.000000,301.510000,0.440000,168.220000
25%,34245.000000,328.000000,672.660000,1.020000,392.760000
50%,68332.000000,662.000000,899.600000,1.460000,562.140000
75%,101679.000000,1009.000000,1222.240000,1.760000,676.530000
max,135943.000000,1339.000000,1754.170000,2.440000,937.770000


In [18]:
bronze_train_df.describe(include="object")

,from,to,flightType,agency,date
count,190321,190321,190321,190321,190321
unique,9,9,3,3,461
top,Florianopolis (SC),Florianopolis (SC),firstClass,Rainbow,2019-09-26
freq,39868,39737,81242,81741,1335


In [19]:
bronze_train_df.nunique()

travelCode    95430
userCode       1335
from              9
to                9
flightType        3
price           490
time             33
distance         35
agency            3
date            461
dtype: int64

### Bronze Pipeline Verification
The `bronze_pipeline.py` is mainly responsible for data validation. It checks for:
- Column presence and order.
- Data types.
Since the bronze data is just a validated version of the raw data, we don't expect to see significant changes in the data itself, but we can be more confident in its quality.

## 3. Silver Data Analysis

In [20]:
silver_train_df = pd.read_parquet(config_silver.SILVER_PROCESSED_DIR / "train.parquet")
silver_train_df.head()

,travel_code,user_code,from_location,to_location,flight_type,price,time,distance,agency,date,year,month,day,day_of_week,day_of_year,week_of_year
0,0,0,Recife (PE),Florianopolis (SC),firstClass,1434.380005,1.76,676.530029,FlyingDrops,2019-09-26,2019,9,26,3,269,39
1,121138,1202,Florianopolis (SC),Natal (RN),firstClass,1315.270020,1.84,709.369995,CloudFy,2019-09-26,2019,9,26,3,269,39
2,132076,1301,Florianopolis (SC),Salvador (BH),premium,1311.380005,2.44,937.770020,CloudFy,2019-09-26,2019,9,26,3,269,39
3,28904,276,Recife (PE),Rio de Janeiro (RJ),economic,908.929993,2.30,885.570007,Rainbow,2019-09-26,2019,9,26,3,269,39
4,88695,877,Aracaju (SE),Natal (RN),firstClass,598.609985,0.46,176.330002,CloudFy,2019-09-26,2019,9,26,3,269,39


In [21]:
silver_train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 190321 entries, 0 to 190320
Data columns (total 16 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   travel_code    190321 non-null  int32         
 1   user_code      190321 non-null  int16         
 2   from_location  190321 non-null  category      
 3   to_location    190321 non-null  category      
 4   flight_type    190321 non-null  category      
 5   price          190321 non-null  float32       
 6   time           190321 non-null  float32       
 7   distance       190321 non-null  float32       
 8   agency         190321 non-null  category      
 9   date           190321 non-null  datetime64[ns]
 10  year           190321 non-null  int16         
 11  month          190321 non-null  int8          
 12  day            190321 non-null  int8          
 13  day_of_week    190321 non-null  int8          
 14  day_of_year    190321 non-null  int16         
 15  

In [22]:
silver_train_df.describe()

,travel_code,user_code,price,time,distance,date,year,month,day,day_of_week,day_of_year,week_of_year
count,190321.000000,190321.000000,190321.000000,190321.000000,190321.000000,190321,190321.000000,190321.000000,190321.000000,190321.000000,190321.000000,190321.000000
mean,67919.412272,667.159567,955.601379,1.418343,545.875244,2020-07-11 06:14:53.299215872,2020.032041,6.449950,15.853080,3.373122,181.345238,26.484939
min,0.000000,0.000000,301.510010,0.440000,168.220001,2019-09-26 00:00:00,2019.000000,1.000000,1.000000,0.000000,1.000000,1.000000
25%,34245.000000,328.000000,672.659973,1.020000,392.760010,2020-02-02 00:00:00,2020.000000,3.000000,8.000000,3.000000,80.000000,12.000000
50%,68332.000000,662.000000,899.599976,1.460000,562.140015,2020-06-26 00:00:00,2020.000000,6.000000,16.000000,3.000000,168.000000,24.000000
75%,101679.000000,1009.000000,1222.239990,1.760000,676.530029,2020-12-10 00:00:00,2020.000000,10.000000,24.000000,4.000000,290.000000,42.000000
max,135943.000000,1339.000000,1754.170044,2.440000,937.770020,2021-07-01 00:00:00,2021.000000,12.000000,31.000000,6.000000,366.000000,53.000000
std,39051.352769,387.624777,362.426453,0.543455,209.208664,NaN,0.638354,3.673053,8.843703,1.653325,112.403690,16.158775


In [23]:
silver_train_df.describe(include=["object", "category"])

,from_location,to_location,flight_type,agency
count,190321,190321,190321,190321
unique,9,9,3,3
top,Florianopolis (SC),Florianopolis (SC),firstClass,Rainbow
freq,39868,39737,81242,81741


In [24]:
cat_silver_df = silver_train_df.select_dtypes(include=["category"])
cat_silver_df.columns

Index(['from_location', 'to_location', 'flight_type', 'agency'], dtype='object')

In [25]:
for col in cat_silver_df.columns:
    print(f"{cat_silver_df[col].cat.categories.to_list()}")

['Aracaju (SE)', 'Brasilia (DF)', 'Campo Grande (MS)', 'Florianopolis (SC)', 'Natal (RN)', 'Recife (PE)', 'Rio de Janeiro (RJ)', 'Salvador (BH)', 'Sao Paulo (SP)']
['Aracaju (SE)', 'Brasilia (DF)', 'Campo Grande (MS)', 'Florianopolis (SC)', 'Natal (RN)', 'Recife (PE)', 'Rio de Janeiro (RJ)', 'Salvador (BH)', 'Sao Paulo (SP)']
['economic', 'firstClass', 'premium']
['CloudFy', 'FlyingDrops', 'Rainbow']


In [26]:
silver_train_df.nunique()

travel_code      95430
user_code         1335
from_location        9
to_location          9
flight_type          3
price              490
time                33
distance            35
agency               3
date               461
year                 3
month               12
day                 31
day_of_week          5
day_of_year        327
week_of_year        53
dtype: int64

In [27]:
silver_train_df.duplicated(keep=False).sum()

0

In [28]:
silver_train_dropped = silver_train_df.drop(
    columns=[
        "travel_code",
        "user_code",
        "date",
        "year",
        "month",
        "day",
        "day_of_week",
        "day_of_year",
        "week_of_year",
    ]
)

In [29]:
silver_train_dropped.sample(10)

,from_location,to_location,flight_type,price,time,distance,agency
141716,Florianopolis (SC),Aracaju (SE),firstClass,1468.030029,2.10,808.849976,CloudFy
94753,Florianopolis (SC),Campo Grande (MS),firstClass,930.340027,1.49,573.809998,FlyingDrops
183716,Salvador (BH),Campo Grande (MS),premium,684.849976,1.36,522.340027,CloudFy
24377,Natal (RN),Recife (PE),economic,429.769989,0.58,222.669998,CloudFy
58554,Florianopolis (SC),Aracaju (SE),firstClass,1574.280029,2.10,808.849976,FlyingDrops
14121,Natal (RN),Sao Paulo (SP),firstClass,1148.780029,0.85,327.549988,Rainbow
83307,Natal (RN),Rio de Janeiro (RJ),economic,774.570007,1.55,595.030029,CloudFy
134554,Recife (PE),Brasilia (DF),premium,573.609985,0.63,242.210007,CloudFy
121451,Florianopolis (SC),Sao Paulo (SP),economic,426.910004,1.46,562.140015,CloudFy
54815,Natal (RN),Recife (PE),premium,579.619995,0.58,222.669998,Rainbow


In [30]:
print(len(silver_train_df))
print(len(silver_train_dropped))

190321
190321


In [31]:
silver_train_dropped.duplicated(keep="first").sum()

189831

In [32]:
190321 - 189831

490

### Silver Pipeline Verification
The `silver_pipeline.py` performs several transformations:
- **Column Renaming and Standardization**: Column names are now in snake_case.
- **Data Type Optimization**: Data types have been optimized (e.g., `date` is now a datetime object).
- **Feature Engineering**: New date-related features have been created (`year`, `month`, `day`, etc.).
- **Duplicate Handling**: Erroneous duplicates have been removed.
By comparing the silver data to the bronze data, we can verify that these transformations have been applied correctly.

## 4. Gold Data Analysis

In [6]:
gold_train_df = pd.read_parquet(config_gold.GOLD_PROCESSED_DIR / "train.parquet")
gold_train_df.head()

,flight_type,from_location,to_location,agency,route,price,time,distance,year,month_sin,month_cos,day_of_week_sin,day_of_week_cos,day_sin,day_cos
0,2.0,5.0,3.0,1.0,43.0,1434.380005,1.76,676.530029,2019.0,-1.0,-1.836970e-16,0.433884,-0.900969,-0.848644,0.528964
1,2.0,3.0,4.0,0.0,27.0,1315.270020,1.84,709.369995,2019.0,-1.0,-1.836970e-16,0.433884,-0.900969,-0.848644,0.528964
2,1.0,3.0,7.0,0.0,30.0,1311.380005,2.44,937.770020,2019.0,-1.0,-1.836970e-16,0.433884,-0.900969,-0.848644,0.528964
3,0.0,5.0,6.0,2.0,45.0,908.929993,2.30,885.570007,2019.0,-1.0,-1.836970e-16,0.433884,-0.900969,-0.848644,0.528964
4,2.0,0.0,4.0,0.0,3.0,598.609985,0.46,176.330002,2019.0,-1.0,-1.836970e-16,0.433884,-0.900969,-0.848644,0.528964


In [ ]:
gold_train_df.sample(10).to_csv("gold_sample.csv")

,flight_type,from_location,to_location,agency,route,price,time,distance,year,month_sin,month_cos,day_of_week_sin,day_of_week_cos,day_sin,day_cos
28991,0.0,2.0,1.0,0.0,17.0,506.559998,0.72,277.700012,2020.0,1.000000e+00,6.123234e-17,-0.433884,-0.900969,-0.790776,-0.612106
13786,1.0,0.0,1.0,0.0,0.0,755.619995,1.11,425.980011,2019.0,-2.449294e-16,1.000000e+00,0.000000,1.000000,-0.101168,-0.994869
15127,1.0,5.0,8.0,2.0,47.0,670.130005,1.26,486.519989,2019.0,-2.449294e-16,1.000000e+00,0.433884,-0.900969,-0.848644,0.528964
30649,2.0,2.0,0.0,0.0,16.0,1581.800049,1.69,650.099976,2020.0,1.000000e+00,6.123234e-17,0.000000,1.000000,-0.201299,0.979530
61680,0.0,1.0,3.0,0.0,10.0,884.940002,1.66,637.559998,2020.0,-8.660254e-01,5.000000e-01,-0.974928,-0.222521,-0.988468,0.151428
81698,1.0,1.0,3.0,2.0,10.0,1222.239990,1.66,637.559998,2021.0,1.000000e+00,6.123234e-17,-0.433884,-0.900969,-0.651372,-0.758758
85952,2.0,3.0,8.0,0.0,31.0,693.510010,1.46,562.140015,2021.0,8.660254e-01,-5.000000e-01,0.433884,-0.900969,-0.968077,-0.250653
50588,1.0,2.0,1.0,2.0,17.0,691.989990,0.72,277.700012,2020.0,-8.660254e-01,-5.000000e-01,-0.974928,-0.222521,0.998717,-0.050649
60378,2.0,0.0,1.0,1.0,0.0,898.039978,1.11,425.980011,2020.0,-8.660254e-01,5.000000e-01,0.433884,-0.900969,0.101168,-0.994869
27813,0.0,2.0,8.0,0.0,23.0,375.510010,0.44,168.220001,2020.0,1.000000e+00,6.123234e-17,-0.433884,-0.900969,0.485302,-0.874347


In [ ]:
gold_train_df.info()

In [ ]:
gold_train_df.describe()

In [ ]:
gold_train_df.nunique()

### Gold Pipeline Verification
The `gold_pipeline.py` applies the final feature engineering and preprocessing steps:
- **Duplcates**: after dropping unique identification columns(travel_code and user_code) there were no erroneous duplicates but duplicate data due to different people boarding the same flight for the same route, these duplicate rows were droped successfully.
- **Imputation**: Missing values have been imputed.
- **Feature Engineering**: Cyclical features and interaction features have been created.
- **Categorical Encoding**: Categorical features have been encoded.
- **Outlier Handling**: Outliers have been handled.
- **Power Transformation and Scaling**: Numerical features have been transformed and scaled.
The gold data is now ready for model training. We can see that all columns are numerical and there are no missing values.

## Get the test indices that were tested with SHAP for prediction by Lightgbm champion model

In [ ]:
test_gold = pd.read_parquet(config_gold.GOLD_PROCESSED_DIR / "test.parquet")

follow the same sampling method with random state used in the shap_explain.py
we used 2000 samples with random_state 42 and the first three instances for that.

In [ ]:
shap_local_instances = test_gold.sample(n=2000, random_state=42).head(3)

In [ ]:
shap_local_instances

lets get the instances reverse transformed for main features so we can make sense of the predictions and features.

In [ ]:
import joblib
from pathlib import Path

scaler_path = config_gold.SCALER_PATH
power_transformer_path = config_gold.POWER_TRANSFORMER_PATH
encoder_path = config_gold.CATEGORICAL_ENCODER_PATH

In [ ]:
# Load the fitted objects
scaler = joblib.load(scaler_path)
power_transformer = joblib.load(power_transformer_path)
encoder = joblib.load(encoder_path)

In [ ]:
df_rev = shap_local_instances.copy()
df_rev = scaler.inverse_transform(df_rev)
df_rev = power_transformer.inverse_transform(df_rev)
ordinal_encoder = [t for t in encoder.preprocessor.transformers_ if t[0] == "ordinal"][
    0
][1]
df_rev["flight_type"] = ordinal_encoder.inverse_transform(
    df_rev[["flight_type"]]
).ravel()

In [ ]:
df_rev[["price", "time", "distance", "flight_type"]]


In [ ]:
shap_local_instances_idx = shap_local_instances.index
shap_local_instances_unprocessed = df_rev[
    ["price", "time", "distance", "flight_type"]
].head()
print(shap_local_instances_unprocessed)

In [ ]:
shap_local_instances.to_csv(core_paths.DOCS_DIR / "shap_local_instances.csv")
shap_local_instances_unprocessed.to_csv(
    core_paths.DOCS_DIR / "shap_local_instances_unprocessed.csv"
)